In [1]:
TRAIN_CSV_PATH = '../C1-P1_Train Dev_fixed/train.csv'
VALID_CSV_PATH = '../C1-P1_Train Dev_fixed/dev.csv'

ORIGINAL_TRAIN_DATA_PATH = '../C1-P1_Train Dev_fixed/C1-P1_Train/' 
ORIGINAL_VALID_DATA_PATH = '../C1-P1_Train Dev_fixed/C1-P1_Dev/' 

TRAIN_DATA_PATH = '../data/train'
VALID_DATA_PATH = '../data/valid'
TEST_DATA_PATH = '../data/test'

In [2]:
import csv
import os
from shutil import copyfile


# make file structure for training dataset
#
with open(TRAIN_CSV_PATH) as csv_file:

    csv_reader = csv.reader(csv_file, delimiter=',')

    line_count = 0
    for row in csv_reader:

        if line_count == 0:
            line_count += 1
            continue  #header

        src_path = os.path.join(ORIGINAL_TRAIN_DATA_PATH, row[0])
        dest_path = os.path.join(TRAIN_DATA_PATH, row[1], row[0])
        if not os.path.isfile(dest_path):
            copyfile(src_path, dest_path)
        
        line_count += 1
        
# make file structure for validation dataset
#
with open(VALID_CSV_PATH) as csv_file:

    csv_reader = csv.reader(csv_file, delimiter=',')

    line_count = 0
    for row in csv_reader:

        if line_count == 0:
            line_count += 1
            continue  #header

        src_path = os.path.join(ORIGINAL_VALID_DATA_PATH, row[0])
        dest_path = os.path.join(VALID_DATA_PATH, row[1], row[0])
        if not os.path.isfile(dest_path):
            copyfile(src_path, dest_path)
        
        line_count += 1



In [3]:
import torch

from torchvision import datasets

import torchvision.transforms as transforms

from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

# how many data per batch to load
batch_size = 20
start_epoch = 0

transform = transforms.Compose([
                                transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomRotation(degrees=(-15, 15)),
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    mean=(0.485, 0.456, 0.406),
                                    std =(0.229, 0.224, 0.225))
                               ])
loaders_transfer = {}
data_transfer = {}

data_transfer['train'] = datasets.ImageFolder(TRAIN_DATA_PATH, transform=transform)
loaders_transfer['train'] = torch.utils.data.DataLoader(data_transfer['train'],
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1)

data_transfer['valid'] = datasets.ImageFolder(VALID_DATA_PATH, transform=transform)
loaders_transfer['valid'] = torch.utils.data.DataLoader(data_transfer['valid'],
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1)
# data_transfer['test'] = datasets.ImageFolder(TEST_DATA_PATH, transform=transform)
# loaders_transfer['test'] = torch.utils.data.DataLoader(data_transfer['test'],
#                                           batch_size=batch_size,
#                                           shuffle=True,
#                                           num_workers=1)

In [4]:
import torchvision.models as models
import torch.nn as nn

# check if CUDA is available
use_cuda = torch.cuda.is_available()

## TODO: Specify model architecture 
model_transfer = models.vgg16(pretrained=True)

for param in model_transfer.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 4096)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.25)),
                          ('fc2', nn.Linear(4096, 3)),
                          ('logsoftmax', nn.LogSoftmax(dim=1))
                          ]))
    
model_transfer.classifier = classifier



if use_cuda:
    model_transfer = model_transfer.cuda()

In [5]:
import torch.optim as optim

criterion_transfer = nn.NLLLoss()
optimizer_transfer = optim.SGD(model_transfer.classifier.parameters(), lr = 0.01)

In [6]:
# train the model

import numpy as np
import time

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        start = time.time()
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders_transfer['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update accumulated training loss
#             train_loss += loss.item()*data.size(0)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders_transfer['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update accumulated validation loss 
#             valid_loss += loss.item()*data.size(0)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

#         train_loss = train_loss/len(loaders_transfer['train'].dataset)
#         valid_loss = valid_loss/len(loaders_transfer['valid'].dataset)
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t time: {:.1f}'.format(
            epoch, 
            train_loss,
            valid_loss,
            time.time() - start
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Validation loss decreased from {:.6f} to {:.6f}. Model was saved'.format(
                valid_loss_min,
                valid_loss
            ))
            
            torch.save(model.state_dict(), 'model_transfer.pt')
            valid_loss_min = valid_loss
    
    # return trained model
    return model

model_transfer = train(12, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

Epoch: 1 	Training Loss: 0.776625 	Validation Loss: 0.804252 	 time: 259.8
Validation loss decreased from inf to 0.804252. Model was saved
Epoch: 2 	Training Loss: 0.640397 	Validation Loss: 0.648182 	 time: 261.1
Validation loss decreased from 0.804252 to 0.648182. Model was saved
Epoch: 3 	Training Loss: 0.601868 	Validation Loss: 0.627028 	 time: 258.7
Validation loss decreased from 0.648182 to 0.627028. Model was saved
Epoch: 4 	Training Loss: 0.581626 	Validation Loss: 0.595220 	 time: 256.2
Validation loss decreased from 0.627028 to 0.595220. Model was saved
Epoch: 5 	Training Loss: 0.559944 	Validation Loss: 0.609914 	 time: 257.1
Epoch: 6 	Training Loss: 0.532254 	Validation Loss: 0.673563 	 time: 257.0
Epoch: 7 	Training Loss: 0.510649 	Validation Loss: 0.640077 	 time: 256.6
Epoch: 8 	Training Loss: 0.491532 	Validation Loss: 0.616345 	 time: 256.0
Epoch: 9 	Training Loss: 0.477155 	Validation Loss: 0.603312 	 time: 256.8
Epoch: 10 	Training Loss: 0.464443 	Validation Loss: 0

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [7]:
# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [8]:
# def test(loaders, model, criterion, use_cuda):

#     # monitor test loss and accuracy
#     test_loss = 0.
#     correct = 0.
#     total = 0.

#     model.eval()
#     for batch_idx, (data, target) in enumerate(loaders['test']):
#         # move to GPU
#         if use_cuda:
#             data, target = data.cuda(), target.cuda()
#         # forward pass: compute predicted outputs by passing inputs to the model
#         output = model(data)
#         # calculate the loss
#         loss = criterion(output, target)
#         # update average test loss 
#         test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
#         # convert output probabilities to predicted class
#         pred = output.data.max(1, keepdim=True)[1]
#         # compare predictions to true label
#         correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
#         total += data.size(0)
            
#     print('Test Loss: {:.6f}\n'.format(test_loss))

#     print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
#         100. * correct / total, correct, total))

# # call test function    
# test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

In [9]:
from PIL import Image
from torch.autograd import Variable

# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = data_transfer['train'].classes

def predict_class_transfer(img_path):
    # load the image and return the predicted breed
    img = Image.open(img_path)

    transform = transforms.Compose([
                                transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()
                               ]) 
    img = transform(img)
    img = img.unsqueeze(0) 

    img = Variable(img)
    if use_cuda:
        img = img.cuda()
        
    prediction = model_transfer(img)  # Returns a Tensor of shape (batch, num class labels)
#     print(prediction)
#     print(class_names)
#     prediction = prediction.data.max(1, keepdim=True)[1]
    prediction = prediction.data.cpu().numpy().argmax()  # Our prediction will be the index of the class label with the largest value.
    prediction = class_names[prediction]
    return prediction 


predict_class_transfer('../data/competition/02186.jpg')

'A'

In [10]:
#Get all test files

from glob import glob
import numpy as np

test_results = []

mango_files = np.array(glob("../data/competition/*"))

for idx, file in enumerate(mango_files):
    _ , filename = os.path.split(file)
    className = predict_class_transfer(file)
    test_results.append([filename, className])
    
test_results[:3]
    

[['02186.jpg', 'A'], ['01873.jpg', 'A'], ['02775.jpg', 'A']]

In [11]:
import csv

with open('results.csv', 'w') as f:

    writer = csv.writer(f)
    
    for row in test_results:
        writer.writerow(row)